In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd

In [ ]:
import numpy as np
import os

# Carga del dataset y preprocesamiento

In [ ]:
# Carga el dataset de AG News
ds, info = tfds.load('ag_news_subset', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/120000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ag_news_subset/incomplete.JX9IZ6_1.0.0/ag_news_subset-train.tfrecord*...: …

Generating test examples...:   0%|          | 0/7600 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/ag_news_subset/incomplete.JX9IZ6_1.0.0/ag_news_subset-test.tfrecord*...:  …

Dataset ag_news_subset downloaded and prepared to /root/tensorflow_datasets/ag_news_subset/1.0.0. Subsequent calls will reuse this data.


In [ ]:
# Convierte el dataset a un formato utilizable
train_ds, test_ds = ds['train'], ds['test']

# Función para convertir un dataset de TensorFlow en una lista de Pandas
def tfds_to_pandas(tfds_dataset):
    texts = []
    labels = []
    for text, label in tfds_dataset:
        texts.append(text.numpy().decode('utf-8'))
        labels.append(label.numpy())
    return pd.DataFrame({'text': texts, 'label': labels})

# Convierte el dataset de entrenamiento y prueba a DataFrames
train_df = tfds_to_pandas(train_ds)
test_df = tfds_to_pandas(test_ds)

In [ ]:

#Unimos todas las filas del dataframe
all_text = ' '.join(train_df['text'].values)
# Caracteres únicos del texto
vocab = sorted(set(all_text))
print(f'{len(vocab)} unique characters')

82 unique characters


In [ ]:
# Creamos una capa de búsqueda que convierte caracteres en sus IDs correspondientes usando el vocabulario proporcionado
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [ ]:
# Creamos una capa que convierte IDs en caracteres usando el vocabulario invertido de ids_from_chars
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)


# Función que convierte secuencias de IDs en texto concatenado usando chars_from_ids
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
# Convierte el texto completo en una secuencia de IDs usando ids_from_chars
all_ids = ids_from_chars(tf.strings.unicode_split(all_text, 'UTF-8'))

# Crea un dataset de TensorFlow a partir de la secuencia de IDs
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

# Longitud de las secuencias a utilizar más 1 para entrada y salida
seq_length = 100

# Agrupa las secuencias en lotes de tamaño seq_length+1, descartando las secuencias incompletas al final
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

# Función para dividir cada secuencia en entrada y salida para el entrenamiento
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

# Aplica la función de división a todas las secuencias en el dataset
dataset = sequences.map(split_input_target)


# Generación del modelo

In [ ]:
# Batch size
BATCH_SIZE = 64


BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [ ]:
# Tamaño del vocabulario en la capa StringLookup
vocab_size = len(ids_from_chars.get_vocabulary())

# Dimensión de embedding
embedding_dim = 256

# Número de unidades en la RNN
rnn_units = 1024

class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

El modelo primero, convierte cada carácter del texto en un ID numérico utilizando capas de búsqueda. Luego, utiliza una capa de embedding para aprender representaciones vectoriales de los caracteres. La capa GRU  procesa secuencialmente estos embeddings para capturar dependencias a largo plazo en el texto. Finalmente, una capa densa predice la probabilidad del próximo carácter en la secuencia, permitiendo así la generación de texto continuo.

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 83) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  21248     
                                                                 
 gru_1 (GRU)                 multiple                  3938304   
                                                                 
 dense_1 (Dense)             multiple                  85075     
                                                                 
Total params: 4044627 (15.43 MB)
Trainable params: 4044627 (15.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Entrenamiento del modelo

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Configuración de Early Stopping
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)

# Ejemplo de predicción
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

# Definición de la función de pérdida
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

# Cálculo de la pérdida media del lote de ejemplo
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

# Conversión de la pérdida media
tf.exp(example_batch_mean_loss).numpy()

# Compilación del modelo
model.compile(optimizer='adam', loss=loss)

# Número de épocas
EPOCHS = 20

# Entrenamiento del modelo con Early Stopping
history = model.fit(dataset, epochs=EPOCHS, callbacks=[early_stopping])

Prediction shape:  (64, 100, 83)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.4197326, shape=(), dtype=float32)
Epoch 1/20
3608/3608 [==============================] - 207s 56ms/step - loss: 1.4341
Epoch 2/20
3608/3608 [==============================] - 205s 56ms/step - loss: 1.1433
Epoch 3/20
3608/3608 [==============================] - 205s 56ms/step - loss: 1.1015
Epoch 4/20
3608/3608 [==============================] - 205s 56ms/step - loss: 1.0852
Epoch 5/20
3608/3608 [==============================] - 204s 56ms/step - loss: 1.0801
Epoch 6/20
3608/3608 [==============================] - 205s 56ms/step - loss: 1.0822
Epoch 7/20
3608/3608 [==============================] - 206s 56ms/step - loss: 1.0904
Epoch 8/20
3608/3608 [==============================] - 206s 57ms/step - loss: 1.1170


En el entrenamiento utilizamos el método de corte EarlyStopping, ya que en pruebas anteriores, la función de pérdida alcanzaba un mínimo en las primeras épocas y luego comenzaba a aunmentar. Por una cuestión de tiempos de uso de la gpu que nos brinda colab,  pudimos probar pocos tamaños de secuencias y otros parámetros para intentar optimizar aún más el modelo.

# Prueba del modelo

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=0.2):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Crear una máscara para evitar que se genere "[UNK]".
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Poner un -inf en cada índice malo.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Emparejar la forma con el vocabulario
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convertir cadenas a IDs de tokens.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Ejecutar el modelo.
    # predicted_logits.shape es [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Usar solo la última predicción.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Aplicar la máscara de predicción: evitar que se genere "[UNK]".
    predicted_logits = predicted_logits + self.prediction_mask

    # Muestrear los logits de salida para generar IDs de tokens.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convertir de IDs de tokens a caracteres
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Devolver los caracteres y el estado del modelo.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
# Lista de semillas para generar noticias
start_words_list = [
    "Breaking news:",
    "In sports today,",
    "The latest economic figures show",
    "Today's weather forecast predicts",
    "In a recent political development,"
]

# Lista para almacenar las noticias generadas
generated_news = []

# Generar noticias para cada semilla en la lista
for start_words in start_words_list:
    states = None
    next_words = tf.constant([start_words])
    result = [next_words]

    for _ in range(400):
        next_words, states = one_step_model.generate_one_step(next_words, states=states)
        result.append(next_words)

    result = tf.strings.reduce_join(result)
    generated_news.append(result.numpy().decode('utf-8'))

# Imprimir todas las noticias generadas
for news in generated_news:
    print(news)
    print("\n" + "_"*80 + "\n")

Breaking news: A senior US military said on Thursday that it has agreed to buy the company #39;s security forces in the country and the United States and the U.S. economy is set to continue to stop the first time in the first time in 19 years. AP - The Canadian and Iraqi forces in the United States and the United States and the United States has announced the first day of the second quarter and the US space age

________________________________________________________________________________

In sports today, as the most powerful typhoon to start Saturday #39;s game against the College State on Saturday. The Consumer Print Hills on Tuesday night to put the start of the season and the Chicago Bears at the College State on Saturday. AP - A second time in the first quarter of 2004, a senior official said on Tuesday. AP - The United States and the United States was sentenced to a second day on Friday as i

________________________________________________________________________________

Th

Aunque el modelo puede generar texto coherente y gramaticalmente correcto, presenta varias repeticiones y redundancias en la información. Las noticias tienden a reciclar frases y estructuras. También, algunas frases carecen de contexto y precisión, resultando en oraciones que no tienen mucho sentido o parecen inconclusas. A pesar de estos problemas, el modelo muestra una capacidad básica para crear titulares y textos en diferentes temas.